# Generating text like Semed Vurgun

In [ ]:
filepath = '/content/Semed_Vurgun_seirleri.txt'
with open(filepath) as f:
    vurgun_text = f.read()

In [ ]:
print(vurgun_text[:200])


CAVANLARA XİTAB
Mənim düşkün könlüm sizdən eləyir
Bu fərz*
 olan təmənnanı, cavanlar.
Unutmayın bəşər sizdən diləyır,
Elm, ədəb, həm irfani, cavanlar.
Bəşər gülşəninin bülbülüsünüz,
Şanlı firqəmizin 


In [ ]:
''.join(sorted(set(vurgun_text.lower())))

'\x01\n !"\'()*,-./0123456789:;<>?[\\]^abcdefghijklmnopqrstuvwxyz§°»äçöüğışə̇–—‘’“”†‡…★'

In [ ]:
text_vec_layer = tf.keras.layers.TextVectorization(split = 'character',
                                                   standardize = 'lower')

text_vec_layer.adapt(vurgun_text)
encoded = text_vec_layer([vurgun_text][0])

In [ ]:
encoded -= 2
vocab_size = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)

In [ ]:
vocab_size

86

In [ ]:
dataset_size

840401

In [ ]:
def to_dataset(sequence, length, seed = None, shuffle = False, batch_size = 32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift = 1, drop_remainder = True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))

    if shuffle:
        ds = ds.shuffle(100_000, seed = seed)

    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [ ]:
length = 100
tf.random.set_seed(42)

train_set = to_dataset(encoded[:1_000_000], length = length, shuffle = True,
                       seed = 42)

valid_set = to_dataset(encoded[1_000_000:1_060_000], length = length)
test_set = to_dataset(encoded[1_060_000:], length = length)

In [ ]:
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = vocab_size, output_dim = 16),
    tf.keras.layers.GRU(128, return_sequences = True),
    tf.keras.layers.Dense(vocab_size, activation = 'softmax')
])

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.Nadam(),
              metrics = ['accuracy'])
model_ckpt = tf.keras.callbacks.ModelCheckpoint('my_shakespeare-model.keras', monitor = 'val_accuracy', save_best_only=True)

history = model.fit(train_set,
                    validation_data = valid_set,
                    epochs = 1,
                    callbacks = [model_ckpt])

   7063/Unknown 1315s 177ms/step - accuracy: 0.3899 - loss: 2.0789

KeyboardInterrupt: 

In [ ]:
vurgun_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),
    model
])

In [ ]:
y_proba = vurgun_model.predict(tf.constant(['el bilir ki sen m']))[0, -1]
y_pred = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_pred + 2]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step


np.str_('ə')

In [ ]:
log_probas = tf.math.log([[0.5, 0.3, 0.2]])
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples = 8)

<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[0, 2, 0, 2, 2, 0, 0, 1]])>

In [ ]:
def next_char(text, temperature = 1):
    text = tf.constant([text])
    y_proba = vurgun_model.predict(text)[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples = 1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [ ]:
def extent_text(text, chars = 50, temperature = 1):
    for _ in range(chars):
        text += next_char(text, temperature)
    return text

In [ ]:
extent_text('salam', temperature = 1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 574ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10

'salamaz.\ntorpaq işçiyalı qapılar olur.\ngoldin, görmədim'

In [ ]:
extent_text('azerbaycan',temperature=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━

'azerbaycan.\nycnə no parçolkm şölümü düşmon vurur doin.\nellər'

## try another one! (will come to this later, tried to use transformer model)

In [ ]:
import tensorflow as tf

vocab_size = 10000
max_length = 50
embed_size = 128
num_heads = 5
ff_dim = 512


# Input layers
encoder_inputs = tf.keras.Input(shape=(None, ), dtype = tf.int32, name='encoder_inputs')
decoder_inputs = tf.keras.Input(shape=(None, ), dtype = tf.int32, name='decoder_inputs')


# Embedding layer
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                                    output_dim = embed_size,
                                                    mask_zero = True)


decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                                    output_dim = embed_size,
                                                    mask_zero = True)

encoder_embeddings = encoder_embedding_layer(encoder_inputs)
decoder_embeddings = decoder_embedding_layer(decoder_inputs)


# Positional Embedding
pos_embedding_layer = tf.keras.layers.Embedding(max_length, embed_size)
positions_encoder = tf.keras.layers.Lambda(lambda x: tf.range(start=0, limit = tf.shape(x)[1], delta=1))(encoder_inputs)
positions_decoder = tf.keras.layers.Lambda(lambda x: tf.range(start=0, limit = tf.shape(x)[1], delta=1))(decoder_inputs)
pos_embed_enc = pos_embedding_layer(positions_encoder)
pos_embed_dec = pos_embedding_layer(positions_decoder)



# Adding positions and token embeddings
encoder_embed = encoder_embeddings + pos_embed_enc
decoder_embed = decoder_embeddings + pos_embed_dec



# Encoder self-attention
encoder_attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size)(encoder_embed, encoder_embed)
encoder_attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoder_embed + encoder_attention)


# Encoder feed-forward
encoder_ff = tf.keras.layers.Dense(ff_dim, activation='relu')(encoder_attention)
encoder_ff = tf.keras.layers.Dense(embed_size)(encoder_ff)
encoder_outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoder_attention + encoder_ff)


# Decoder self-attention
casual_mask = tf.keras.layers.Lambda(
    lambda x: tf.linalg.band_part(tf.ones((tf.shape(x)[1], tf.shape(x)[1])), -1, 0)
)(decoder_inputs)

decoder_attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, \
                                                       key_dim=embed_size)(decoder_embed, decoder_embed, attention_mask=casual_mask)
decoder_attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(decoder_embed + decoder_attention)



# Encoder Decoder cross-attention
cross_attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size)(decoder_attention, encoder_outputs, encoder_outputs)
decoder_cross = tf.keras.layers.LayerNormalization(epsilon=1e-6)(decoder_attention + cross_attention)


# Decoder feed-forward
decoder_ff = tf.keras.layers.Dense(ff_dim, activation='relu')(decoder_cross)
decoder_ff = tf.keras.layers.Dense(embed_size)(decoder_ff)
decoder_outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(decoder_cross + decoder_ff)


# Final output-layer
output_logits = tf.keras.layers.Dense(vocab_size, activation='softmax')(decoder_outputs)
transformer = tf.keras.Model([encoder_inputs, decoder_inputs], output_logits)

In [ ]:
transformer.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='nadam',
    metrics=['accuracy']
)

In [ ]:
filepath = '/content/Semed_Vurgun_seirleri.txt'
with open(filepath) as f:
    vurgun_text = f.read()

In [ ]:
text_vec_layer = tf.keras.layers.TextVectorization(split = 'character',
                                                   standardize = 'lower')

text_vec_layer.adapt(vurgun_text)
encoded = text_vec_layer([vurgun_text][0])

In [ ]:
encoded -= 2
vocab_size = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)

In [ ]:
def to_dataset(sequence, length, seed = None, shuffle = False, batch_size = 32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift = 1, drop_remainder = True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))

    if shuffle:
        ds = ds.shuffle(100_000, seed = seed)

    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [ ]:
length = 100
tf.random.set_seed(42)

train_set = to_dataset(encoded[:1_000_000], length = length, shuffle = True,
                       seed = 42)

valid_set = to_dataset(encoded[1_000_000:1_060_000], length = length)
test_set = to_dataset(encoded[1_060_000:], length = length)

In [ ]:
for i in to_dataset(encoded[:1000], length=length, seed=42).take(1):
  print(i[1])

tf.Tensor(
[[30  1 28 ...  2 24  2]
 [ 1 28  1 ... 24  2  4]
 [28  1  3 ...  2  4  0]
 ...
 [23 26  3 ... 13 16  0]
 [26  3  6 ... 16  0 27]
 [ 3  6 17 ...  0 27  2]], shape=(32, 100), dtype=int64)


In [ ]:
tf.random.set_seed(42)

history = transformer.fit((i for i, j in train_set.take(10)),
                    # validation_data = valid_set,
                    epochs = 5)

ValueError: When passing a Python generator to a Keras model, the generator must return a tuple, either (input,) or (inputs, targets) or (inputs, targets, sample_weights). Received: [[ 4 19 19 ... 17  3 11]
 [ 0 14  6 ...  3  0  3]
 [ 1  4  9 ...  3  0 25]
 ...
 [11  1 27 ... 32  6 15]
 [18  0 20 ...  2 12  5]
 [ 2 10  0 ...  3  2  0]]

In [ ]:
vurgun_model_with_transformer = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),
    transformer
])

In [ ]:
y_proba = vurgun_model_with_transformer.predict(tf.constant(['el bilir ki sen m']))[0, -1]
y_pred = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_pred + 2]

In [ ]:
def next_char(text, temperature = 1):
    text = tf.constant([text])
    y_proba = vurgun_model_with_transformer.predict(text)[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples = 1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [ ]:
def extent_text(text, chars = 50, temperature = 1):
    for _ in range(chars):
        text += next_char(text, temperature)
    return text

In [ ]:
extent_text('salam')